In [3]:
#Importar librerías y archivo
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
games_df=pd.read_csv('games.csv')


In [4]:
#Preprocesamiento de datos: preprocesar los datos para poder calcular la similitud del coseno

# Seleccionar las columnas relevantes para el cálculo de similitud
selected_columns = ['id', 'Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling' , 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']

# Filtrar el DataFrame para incluir solo las columnas seleccionadas
games_subset = games_df[selected_columns]

# Normalizar las columnas de género
scaler = StandardScaler()
games_normalized = scaler.fit_transform(games_subset.iloc[:, 1:])

# Crear un nuevo DataFrame con los datos normalizados
games_normalized_df = pd.DataFrame(games_normalized, columns=games_subset.columns[1:])

# Combinar el ID con el DataFrame normalizado
games_normalized_df['id'] = games_df['id']

#Calcular la similitud del coseno: usar la función cosine_similarity de scikit-learn

# Eliminar la columna 'id' temporalmente para calcular la similitud
game_ids = games_normalized_df['id']
games_normalized_df = games_normalized_df.drop(columns=['id'])

# Calcular la matriz de similitud del coseno
cosine_sim_matrix = cosine_similarity(games_normalized_df, games_normalized_df)

# Convertir la matriz de similitud en un DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=game_ids, columns=game_ids)

# Restablecer la columna 'id'
games_normalized_df['id'] = game_ids

In [5]:
#Función de recomendación: función que toma el id de un juego y recomienda 5 juegos similares basados en la similitud del coseno.
def recommend_games(game_id):
    # Encuentra el nombre del juego en base al id
    game_title = games_df[games_df['id'] == game_id]['title'].values[0]

    # Obtiene las puntuaciones de similitud del juego específico
    game_similarity_scores = cosine_sim_df[game_id]

    # Ordena los juegos por similitud descendente
    recommended_game_ids = game_similarity_scores.sort_values(ascending=False).index[1:6]

    # Obtiene los títulos de los 5 juegos recomendados
    recommended_games = games_df[games_df['id'].isin(recommended_game_ids)]['title']

    return recommended_games

In [17]:
print('Los juegos similares a Tom Raider son:\n', recommend_games(8000))
print('Los juegos similares a Ironbound son:\n', recommend_games(643980))

Los juegos similares a Tom Raider son:
 1719     Assassin’s Creed® IV Black Flag™ - Time saver:...
1730          Saints Row IV - Hey Ash Whatcha Playin? Pack
11069         Outcast - Second Contact Original Soundtrack
11407             Instinct Coin &amp; Madhouse Mode Unlock
22891                                 Way of the Samurai 4
Name: title, dtype: object
Los juegos similares a Ironbound son:
 5771     Immortal Empire - Starter Pack 2
9952             Labyrinth - Starter Pack
20356    Immortal Empire - Mercenary Pack
23234                Vulture for Slash'EM
27112           The Banner Saga: Factions
Name: title, dtype: object
